In [5]:
import pandas as pd
from copy import deepcopy


In [10]:
getListChart = pd.read_excel("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/Du lieu api/getListChart_202309121406.xlsx")

In [11]:
service_id_all = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/SERVICE_GBTD_202309051334.csv")
service_id_list = list(set(service_id_all['SERVICE_ID']))
service_id_dictionary = {u: list(set(service_id_all[service_id_all['SERVICE_ID']==u]['SERVICE_NAME']))[0] for u in service_id_list}

In [12]:
getListChartName = deepcopy(getListChart)
getListChartName.drop(columns=[], inplace=True)
getListChartName['SERVICE_NAME'] = getListChartName['SERVICEID'].apply(lambda i: service_id_dictionary[i] if i in service_id_dictionary.keys() else "")
getListChartName

,SERVICEID,AREACODE,CYCLETIME,TYPECHART,CHARTID,TITLECHART,TYPE_CHART,SERVICE_NAME
0,157,A076,NAM,Biểu đồ đường,8340,AGG_DTDV CỐ ĐỊNH_TỪNG NĂM,LINE,Doanh thu cố định
1,20212510,A076,NAM,Biểu đồ đường,8368,AGG_DTDV NHÓM VDS_TỪNG NĂM,LINE,DT dịch vụ nhóm DV của VDS
2,20214132,A076,NAM,Biểu đồ đường,8357,AGG_DTDV NHÓM VTS_TỪNG NĂM,LINE,DT dịch vụ nhóm DV của VTS
3,20214133,A076,NAM,Biểu đồ đường,8326,AGG_DTDV NHÓM VTT_TỪNG NĂM,LINE,DT dịch vụ nhóm DV của VTT
4,5066,A076,NAM,Biểu đồ đường,8348,AGG_DTDV SME_TỪNG NĂM,LINE,Doanh thu dịch vụ SME
...,...,...,...,...,...,...,...,...
35508,1314,STL,THANG,Biểu đồ cột,5332,rpm thoại Xu hướng tháng,COLUMN,RPM thoại
35509,1144,TD,THANG,Biểu đồ cột chồng,21918,test cột chồng,STACK,DOANH THU DỊCH VỤ (trừ hubbing)
35510,143,TD,THANG,Biểu đồ cột chồng,21918,test cột chồng,STACK,DOANH THU DỊCH VỤ (chi tiết)
35511,1202103,VDS,THANG,Biểu đồ đường,22573,ĐIỂM CHẤP NHẬN THANH TOÁN PSGD_TỪNG THÁNG,LINE,Số lượng điểm Chấp nhận thanh toán PSGD lũy kế


In [184]:
def remove_donvi(title_chart):
    features = title_chart.split("_")
    if " " not in features[0] and features[0] not in ["company", 'info', 'test', 'rank', 'rada', 'table', 'tooltip', '21', 'dgsdf', 'khdn', 'dt-cp-ln', 'arpu', 'mou', 'dou']:
        output =  features[0]
        output2 = "_".join(features[1:])
    elif features[0] in ['công ty mẹ', 'tập đoàn', 'ban cntt', 'khối tỉnh/tp', 'công đoàn', 'viettel tỉnh/tp', 'viettel tỉnh tp']:
        output = features[0]
        output2 = "_".join(features[1:])
    elif len(features) == 4 and " " not in features[2]:
        output = features[2]
        output2 = "_".join(features[:2]+features[3:])
    else:
        output = ""
        output2 = "_".join(features[0:])
    return output, output2
    

def remove_time(title_chart_rm_donvi):
    features = title_chart_rm_donvi.split("_")
    if len(features) == 1:
        features = "_".join(features)
        return features, ""
    output2 = ""
    output = deepcopy(features)
    for feature in features:
        for i in ['ngay', 'thang', 'nam', 'quy', 'ngày', 'tháng', 'năm', 'quý', 'tết', 'qúy']:
            if i in feature:
                output2 = feature
                output.remove(feature)
                break
    
    output = "_".join(output)
    return output, output2

def remove_redundant(title_chart_re_donvi_time):
    features = title_chart_re_donvi_time.split("_")
    output2 = ""
    output = deepcopy(features)
    if len(features) > 1:
        for feature in features:
            feature2 = feature.strip()
            if feature2 in ['line', 
                           'theo hướng', 
                           'xu huong', 
                           'theo cơ cấu', 
                           'cơ cấu từng khối', 
                           'cơ cấu cơ cấu từng khối', 
                           'cơ cấu dịch vụ', 
                           'cơ cấu đơn vị (lk kệ thống)',
                           'cơ cấu khối',
                           'các đơn vị',
                           'cơ cấu đơn vị',
                           'từng đơn vị',
                           'các đơn vị con nội bộ',
                           'cơ cấu %',
                           'theo loại thuê bao',
                           ] \
                or "theo hạ tầng" in feature:
                output2 = feature2
                output.remove(feature)
                break

            if 'test' in feature:
                output2 = feature
                output.remove(feature)
    output = "_".join(output)
    return output, output2

def check_final_name(row):
    if row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'] in row['TYPECHART']:
        return ""
    
    if row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'].isnumeric():
        return ""
    
    list_check2 = ['infomain', 'info_main', 'check', 'làm tròn', 'thoa', 'hà', 'copy', 'mai']
    for item in list_check2:
        if item in row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT']:
            return ""

    if row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'][0].isnumeric() \
        and row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'][1] in [' ', '.']:
        output = row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'][2:].strip()
    else:
        output = row['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'].strip()

    return output

def check_final_name_for_input(output):
    if len(output.split("_")) > 1:
        output = " ".join([i.strip() for i in output.split("_")])
    return output
    

In [186]:
chart_dept_for_title = deepcopy(getListChartName)
chart_dept_for_title = chart_dept_for_title[['AREACODE', 'CYCLETIME', 'TITLECHART', 'SERVICE_NAME', 'TYPECHART']]
chart_dept_for_title = chart_dept_for_title[~chart_dept_for_title['TITLECHART'].str.contains('test')]
chart_dept_for_title = chart_dept_for_title[~chart_dept_for_title['TITLECHART'].str.contains('copy')]
chart_dept_for_title = chart_dept_for_title.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
chart_dept_for_title['CHECK_LEN'] = chart_dept_for_title['TITLECHART'].apply(lambda x: len(x.split('_')))
chart_dept_for_title['DONVI_FEATURE'] = chart_dept_for_title['TITLECHART'].apply(lambda x: remove_donvi(x)[0])
chart_dept_for_title['TITLE_CHART_RM_DONVI'] = chart_dept_for_title['TITLECHART'].apply(lambda x: remove_donvi(x)[1])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[0])
chart_dept_for_title.drop(columns=['TITLE_CHART_RM_DONVI', 'CHECK_LEN'], inplace=True)
chart_dept_for_title['REDUNDANT'] = chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'].apply(lambda x: remove_redundant(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME_REDUNDANT'] = chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'].apply(lambda x: remove_redundant(x)[0])
chart_dept_for_title['FINAL_TITLE'] = chart_dept_for_title.apply(lambda x: check_final_name(x), axis=1)
chart_dept_for_title['FINAL_TITLE_FOR_INPUT'] = chart_dept_for_title['FINAL_TITLE'].apply(check_final_name_for_input)
chart_dept_for_title


,AREACODE,CYCLETIME,TITLECHART,SERVICE_NAME,TYPECHART,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME,REDUNDANT,TITLE_CHART_RM_DONVI_TIME_REDUNDANT,FINAL_TITLE,FINAL_TITLE_FOR_INPUT
0,a076,nam,agg_dtdv cố định_từng năm,doanh thu cố định,biểu đồ đường,agg,từng năm,dtdv cố định,,dtdv cố định,dtdv cố định,dtdv cố định
1,a076,nam,agg_dtdv nhóm vds_từng năm,dt dịch vụ nhóm dv của vds,biểu đồ đường,agg,từng năm,dtdv nhóm vds,,dtdv nhóm vds,dtdv nhóm vds,dtdv nhóm vds
2,a076,nam,agg_dtdv nhóm vts_từng năm,dt dịch vụ nhóm dv của vts,biểu đồ đường,agg,từng năm,dtdv nhóm vts,,dtdv nhóm vts,dtdv nhóm vts,dtdv nhóm vts
3,a076,nam,agg_dtdv nhóm vtt_từng năm,dt dịch vụ nhóm dv của vtt,biểu đồ đường,agg,từng năm,dtdv nhóm vtt,,dtdv nhóm vtt,dtdv nhóm vtt,dtdv nhóm vtt
4,a076,nam,agg_dtdv sme_từng năm,doanh thu dịch vụ sme,biểu đồ đường,agg,từng năm,dtdv sme,,dtdv sme,dtdv sme,dtdv sme
...,...,...,...,...,...,...,...,...,...,...,...,...
35506,y029,thang,ybi_tiêu dùng thực di động_từng tháng,tiêu dùng thực di động (tt+ts),line_dashed chart,ybi,từng tháng,tiêu dùng thực di động,,tiêu dùng thực di động,tiêu dùng thực di động,tiêu dùng thực di động
35507,stl,thang,rpm thoại xu hướng tháng,rpm thoại,biểu đồ đường,,,rpm thoại xu hướng tháng,,rpm thoại xu hướng tháng,rpm thoại xu hướng tháng,rpm thoại xu hướng tháng
35508,stl,thang,rpm thoại xu hướng tháng,rpm thoại,biểu đồ cột,,,rpm thoại xu hướng tháng,,rpm thoại xu hướng tháng,rpm thoại xu hướng tháng,rpm thoại xu hướng tháng
35511,vds,thang,điểm chấp nhận thanh toán psgd_từng tháng,số lượng điểm chấp nhận thanh toán psgd lũy kế,biểu đồ đường,,từng tháng,điểm chấp nhận thanh toán psgd,,điểm chấp nhận thanh toán psgd,điểm chấp nhận thanh toán psgd,điểm chấp nhận thanh toán psgd


In [18]:
chart_dept_for_title = deepcopy(getListChartName)
chart_dept_for_title = chart_dept_for_title.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
chart_dept_for_title['CHECK_LEN'] = chart_dept_for_title['TITLECHART'].apply(lambda x: len(x.split("_")))
chart_dept_for_title['DONVI_FEATURE'] = chart_dept_for_title['TITLECHART'].apply(lambda x: remove_donvi(x)[0])
chart_dept_for_title['TITLE_CHART_RM_DONVI'] = chart_dept_for_title['TITLECHART'].apply(lambda x: remove_donvi(x)[1])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[0])
chart_dept_for_title.drop(columns=['TITLE_CHART_RM_DONVI'], inplace=True)